In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import copy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

has_mps = torch.backends.mps.is_built()
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

Using device: cpu


In [ ]:
##

# Do Feature Engineering and 69 Others.... Here

In [3]:
#Do X=df.drop("target",axis=1),y=df["target"]. then Label Encoder.
# then check nan value,variance check(most important)

In [ ]:
torch.manual_seed(42)
X_tensor=torch.tensor(X.values,dtype=torch.float32,device=device)
device
y_tensor=torch.tensor(y.values,dtype=torch.float32,device=device).view(-1,1)

kf=KFold(n_splits=5,shuffle=True,random_state=42)
kf

patience=10

fold=0

for train_index,test_index in kf.split(X_tensor):
    fold+=1
    print(f"Fold {fold}")
    X_train, X_test= X_tensor[train_index], X_tensor[test_index]
    y_train, y_test= y_tensor[train_index], y_tensor[test_index]



    train_dataset=TensorDataset(X_train,y_train)
    train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True)


    model=nn.Sequential(
        nn.Linear(12,20),
        nn.ReLU(),
        nn.Linear(20,20),
        nn.ReLU(),
        nn.Linear(20,1),
        nn.ReLU()
    )
    model = torch.compile(model,backend="aot_eager").to(device)



    loss_fn=nn.MSELoss()
    optimizer=torch.optim.Adam(params=model.parameters(),lr=0.01)
    # Early Stopping variables
    best_loss = float('inf')
    early_stopping_counter = 0

    Epochs=500
    
    # Training loop
    EPOCHS = 500
    epoch = 0
    done = False
    es = EarlyStopping()


    while not done and epoch<EPOCHS:
        epoch += 1
        model.train()
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.inference_mode():
            val_output = model(X_test)
            val_loss = loss_fn(val_output, y_test)

        if es(model, val_loss):
            done = True

    print(f"Epoch {epoch}/{EPOCHS}, Validation Loss: "
      f"{val_loss.item()}, {es.status}")

# Final evaluation
model.eval()
with torch.inference_mode():
    oos_pred = model(X_test)
score = torch.sqrt(loss_fn(oos_pred, y_test)).item()
print(f"Fold score (RMSE): {score}")

# Or use this ...same just looks more beautiful

In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold

# Set random seed for reproducibility
torch.manual_seed(42)

# Convert data to tensors and move to device
X_tensor = torch.tensor(X.values, dtype=torch.float32, device=device)
y_tensor = torch.tensor(y.values, dtype=torch.float32, device=device).view(-1, 1)

# Initialize K-Fold cross validator
kf = KFold(n_splits=5, shuffle=True, random_state=42)
patience = 10

# Initialize fold counter
fold = 0

for train_index, test_index in kf.split(X_tensor):
    fold += 1
    print(f"\n=== Fold {fold} ===")
    
    # Split data into train and test sets
    X_train, X_test = X_tensor[train_index], X_tensor[test_index]
    y_train, y_test = y_tensor[train_index], y_tensor[test_index]
    
    # Create DataLoader for training
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    # Define model architecture
    model = nn.Sequential(
        nn.Linear(12, 20),
        nn.ReLU(),
        nn.Linear(20, 20),
        nn.ReLU(),
        nn.Linear(20, 1),
        nn.ReLU()
    )
    
    # Compile and move model to device
    model = torch.compile(model, backend="aot_eager").to(device)
    
    # Initialize loss function and optimizer
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
    
    # Initialize early stopping
    early_stopping = EarlyStopping()
    EPOCHS = 500
    epoch = 0
    done = False
    
    # Training loop
    while not done and epoch < EPOCHS:
        epoch += 1
        
        # Training phase
        model.train()
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()
        
        # Validation phase
        model.eval()
        with torch.inference_mode():
            val_output = model(X_test)
            val_loss = loss_fn(val_output, y_test)
        
        # Check early stopping condition
        if early_stopping(model, val_loss):
            done = True
        
        # Print progress
        if epoch % 50 == 0 or done:
            print(f"Epoch {epoch:3d}/{EPOCHS}, Validation Loss: {val_loss.item():.4f}, "
                  f"{early_stopping.status}")
    
    # Final evaluation
    model.eval()
    with torch.inference_mode():
        oos_pred = model(X_test)
        score = torch.sqrt(loss_fn(oos_pred, y_test)).item()
    
    print(f"\nFold {fold} score (RMSE): {score:.4f}")